In [564]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.metrics import accuracy_score, classification_report
from sklearn.neural_network import MLPRegressor, MLPClassifier
from sklearn.metrics import accuracy_score, mean_squared_error
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import resample
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, TransformerMixin
#from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.impute import SimpleImputer
from imblearn.pipeline import Pipeline
from imblearn import FunctionSampler

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier
from sklearn.linear_model import SGDClassifier

from sklearn.metrics import precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE, BorderlineSMOTE, KMeansSMOTE, SVMSMOTE, ADASYN, RandomOverSampler
from sklearn.feature_selection import SelectKBest, f_classif, chi2, SelectPercentile, SelectFromModel, SequentialFeatureSelector, SelectFromModel
from sklearn.decomposition import PCA
from scipy import stats
from imblearn.base import SamplerMixin

In [565]:
class ManualFeatureSelectorTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        print("fit - ManualFeatureSelectorTransformer - begin", X.shape)
        print("fit - ManualFeatureSelectorTransformer - end", X.shape)
        return self

    def transform(self, X):
        print("transform - ManualFeatureSelectorTransformer", X.shape)
        #X.drop('Radiacao Global', axis=1, inplace=True)
        X.drop('Cidade', axis=1, inplace=True)
        X.drop('Codigo', axis=1, inplace=True)
        print("transform - ManualFeatureSelectorTransformer - end", X.shape)
        return X

In [566]:
class ImputerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.imputer_numeric = SimpleImputer(strategy='mean')
        self.imputer_categorical = SimpleImputer(strategy='most_frequent')

    def fit(self, X, y=None):
        print("fit - ImputerTransformer - begin", X.shape)
        numeric_cols = X.select_dtypes(include=['number']).columns
        categorical_cols = X.select_dtypes(include=['object']).columns

        self.imputer_numeric.fit(X[numeric_cols])
        self.imputer_categorical.fit(X[categorical_cols])
        print("fit - ImputerTransformer - end", X.shape)
        return self

    def transform(self, X):
        print("transform - ImputerTransformer - begin", X.shape)
        numeric_cols = X.select_dtypes(include=['number']).columns
        categorical_cols = X.select_dtypes(include=['object']).columns
        X[numeric_cols] = self.imputer_numeric.transform(X[numeric_cols])
        X[categorical_cols] = self.imputer_categorical.transform(X[categorical_cols])
        print("transform - ImputerTransformer - end", X.shape)
        return X

In [567]:
class DateTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        print("fit - DateTransformer - begin", X.shape)
        print("fit - DateTransformer - end", X.shape)
        return self

    def transform(self, X):
        print("transform - DateTransformer", X.shape)
        X['Data'] = pd.to_datetime(X['Data'])
        X['Ano'] = X['Data'].dt.year
        X['Mes'] = X['Data'].dt.month
        X['Dia'] = X['Data'].dt.day
        X.drop('Data', axis=1, inplace=True)
        print("transform - DateTransformer - end", X.shape)
        return X

In [568]:
class OneHotEncoderTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoder = OneHotEncoder(sparse=False)

    def fit(self, X, y=None):
        print("fit - OneHotEncoderTransformer - begin", X.shape)
        self.encoder.fit(X[['Codigo']])
        print("fit - OneHotEncoderTransformer - end", X.shape)
        return self

    def transform(self, X):
        print("transform - OneHotEncoderTransformer - begin", X.shape)
        X.reset_index(drop=True, inplace=True)
        encoded_data = self.encoder.transform(X[['Codigo']])
        encoded_df = pd.DataFrame(encoded_data, columns=self.encoder.get_feature_names_out(['Codigo']))
        X = pd.concat([X.drop(['Codigo'], axis=1), encoded_df], axis=1)
        print("transform - OneHotEncoderTransformer - end", X.shape)
        return X

In [569]:
class RobustScalerTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = RobustScaler()

    def fit(self, X, y=None):
        print("fit - RobustScalerTransformer - begin", X.shape)
        self.scaler.fit(X[['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia','Radiacao Global']])
        print("fit - RobustScalerTransformer - end", X.shape)
        return self

    def transform(self, X):
        print("transform - RobustScalerTransformer - begin", X.shape)
        X[['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima','Umidade Minima',
      'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 'Direcao Vento',
      'Rajada Maxima de Vento','Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia', 'Radiacao Global']] = self.scaler.transform(
          X[['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia', 'Radiacao Global']])
        print("transform - RobustScalerTransformer - end", X.shape)
        return X

In [570]:
def removeLocalOutlier(X, y):
    print("fit_resample - LocalOutlierTransformer - begin", X.shape)
    lof = LocalOutlierFactor(contamination=0.025)
    outlier_mask = lof.fit_predict(X) != -1
    X = X[outlier_mask]
    y = y[outlier_mask]
    print("fit_resample - LocalOutlierTransformer - end",X.shape)
    return X, y

In [586]:
def handleOutliersZIndex(X, threshold = 3):
    print("handleOutliersZIndex - begin", X.shape)
    columns = ['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Radiacao Global']
    
    for column in columns:
        non_zero_mask = (X[column].notna())

        nz_X = X[non_zero_mask]
        z_scores = np.abs(stats.zscore(nz_X[column]))
        outliers_mask = (z_scores > threshold)

        nz_X.loc[outliers_mask, column] = nz_X[(~outliers_mask)][column].mean()

        X.loc[non_zero_mask, column] = nz_X[column]
    
    column = 'Precipitacao Total'
    non_zero_mask = (X[column] != 0) & (X[column].notna())

    nz_X = X[non_zero_mask]
    z_scores = np.abs(stats.zscore(nz_X[column]))
    outliers_mask = (z_scores > threshold)

    nz_X.loc[outliers_mask, column] = nz_X[(~outliers_mask)][column].mean()

    X.loc[non_zero_mask, column] = nz_X[column]

    print("handleOutliersZIndex - end", X.shape)
    return X

In [572]:
def get_mlp(data_x, data_y):
    mlp = MLPClassifier(hidden_layer_sizes=(10,20,20,10), activation='relu', max_iter=15, random_state=42)
    mlp.fit(data_x, data_y)
    return mlp

In [573]:
def get_dt(data_x, data_y):
    dt = DecisionTreeClassifier(class_weight='balanced', splitter='best', max_depth=None, criterion='gini')
    dt.fit(data_x, data_y)
    return dt

In [574]:
def get_rf(data_x, data_y):
    rf = RandomForestClassifier(class_weight='balanced', max_depth=15, n_estimators=20, random_state=42)
    rf.fit(data_x, data_y)
    return rf

In [575]:
def get_sgd(data_x, data_y):
    sgd = SGDClassifier(max_iter=3000, tol=1e-3, class_weight='balanced', random_state=42)
    sgd.fit(data_x, data_y)
    return sgd

In [576]:
def get_logistic_regression(data_x, data_y):
    lr = LogisticRegression(class_weight='balanced', max_iter=2000, random_state=42)
    lr.fit(data_x, data_y)
    return lr

In [577]:
def get_bagging(data_x, data_y):
   base_classifier = DecisionTreeClassifier(class_weight='balanced', splitter='best', max_depth=15, criterion='gini', random_state=42)
   #base_classifier = DecisionTreeClassifier(splitter='best', max_depth=20, criterion='gini', random_state=42, min_samples_split=50)
   bagging = BaggingClassifier(base_estimator=base_classifier, n_estimators=20, random_state=42)
   bagging.fit(data_x, data_y)
   return bagging

In [578]:
def get_linear_svc(data_x, data_y):
    linear_svc = LinearSVC(class_weight='balanced', random_state=42)
    linear_svc.fit(data_x, data_y)
    return linear_svc

In [579]:
def get_knn(data_x, data_y):
    knn = KNeighborsClassifier(class_weight='balanced', n_neighbors=20)
    knn.fit(data_x, data_y)
    return knn

In [580]:
def get_nb(data_x, data_y):
    nb = BernoulliNB()
    nb.fit(data_x, data_y)
    return nb

In [581]:
def get_ada(data_x, data_y):
    ada = AdaBoostClassifier(n_estimators=100, algorithm="SAMME", random_state=0)
    ada.fit(data_x, data_y)
    return ada

In [582]:
def plot_result_scikit(model, data_x, data_y):
    y_pred = model.predict(data_x)
    accuracy = accuracy_score(data_y, y_pred),
    FP = np.sum((y_pred == 'Sim') & (data_y == 'Nao'))
    FN = np.sum((y_pred == 'Nao') & (data_y == 'Sim'))
    VP = np.sum((y_pred == 'Sim') & (data_y == 'Sim'))
    VN = np.sum((y_pred == 'Nao') & (data_y == 'Nao'))

    y_true  = (data_y == 'Sim').astype(int)
    y_pred  = (y_pred == 'Sim').astype(int)

    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"Accuracy: {accuracy} | Precision:{precision} | Recall:{recall} | F1-score:{f1} | FP:{FP} | FN:{FN} | VP:{VP} | VN:{VN}")

In [587]:
data = pd.read_csv('data.csv', delimiter=';')

print(data.shape[0])
data = data.dropna(subset=['Vai Chover Amanha'])
#data = data.dropna()
data.reset_index(drop=True, inplace=True)
print(data.shape[0])

X = data.drop('Vai Chover Amanha', axis=1)
y = data['Vai Chover Amanha']

raw_X_train, X_test, raw_y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=60)


pipeline = Pipeline(steps=[
    ('manual',ManualFeatureSelectorTransformer()),
    ('imputer',ImputerTransformer()),
    ('date', DateTransformer()),
    #('oht', OneHotEncoderTransformer()),
    #('outliers', LocalOutlierTransformer()),
    ('scaler', RobustScalerTransformer()),
    #('over_sampler', RandomOverSampler(random_state=42)),
    #('model', bagging = BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight='balanced', splitter='best', max_depth=15, criterion='gini', random_state=42), n_estimators=20, random_state=42)),
])

X_train = pipeline.fit_transform(handleOutliersZIndex(raw_X_train.copy(),4), raw_y_train.copy())
y_train = raw_y_train.copy()

X_test = pipeline.transform(X_test.copy())

os = RandomOverSampler(random_state=42)
X_train, y_train = os.fit_resample(X_train.copy(), y_train.copy())

scikit_model = get_bagging(X_train.copy(), y_train.copy())
print("bagging")
print("training:")
plot_result_scikit(scikit_model, pipeline.transform(raw_X_train.copy()), raw_y_train.copy()) 
print("over sampled training:")
plot_result_scikit(scikit_model, X_train.copy(), y_train.copy()) 
print("test:")
plot_result_scikit(scikit_model, X_test.copy(),y_test.copy())

scikit_model = get_rf(X_train.copy(), y_train.copy())
print("rf")
print("training:")
plot_result_scikit(scikit_model, pipeline.transform(raw_X_train.copy()), raw_y_train.copy()) 
print("over sampled training:")
plot_result_scikit(scikit_model, X_train.copy(), y_train.copy()) 
print("test:")
plot_result_scikit(scikit_model, X_test.copy(),y_test.copy())

scikit_model = get_ada(X_train, y_train)
print("ada")
print("training:")
plot_result_scikit(scikit_model, pipeline.transform(raw_X_train.copy()), raw_y_train.copy()) 
print("over sampled training:")
plot_result_scikit(scikit_model, X_train.copy(), y_train.copy()) 
print("test:")
plot_result_scikit(scikit_model, X_test.copy(),y_test.copy())

scikit_model = get_nb(X_train, y_train)
print("nb")
print("training:")
plot_result_scikit(scikit_model, pipeline.transform(raw_X_train.copy()), raw_y_train.copy()) 
print("over sampled training:")
plot_result_scikit(scikit_model, X_train.copy(), y_train.copy()) 
print("test:")
plot_result_scikit(scikit_model, X_test.copy(),y_test.copy())

scikit_model = get_mlp(X_train, y_train)
print("mlp")
print("training:")
plot_result_scikit(scikit_model, pipeline.transform(raw_X_train.copy()), raw_y_train.copy()) 
print("over sampled training:")
plot_result_scikit(scikit_model, X_train.copy(), y_train.copy()) 
print("test:")
plot_result_scikit(scikit_model, X_test.copy(),y_test.copy())

248121
232731
handleOutliersZIndex - begin (186184, 22)
handleOutliersZIndex - end (186184, 22)
fit - ManualFeatureSelectorTransformer - begin (186184, 22)
fit - ManualFeatureSelectorTransformer - end (186184, 22)
transform - ManualFeatureSelectorTransformer (186184, 22)
transform - ManualFeatureSelectorTransformer - end (186184, 20)
fit - ImputerTransformer - begin (186184, 20)
fit - ImputerTransformer - end (186184, 20)
transform - ImputerTransformer - begin (186184, 20)
transform - ImputerTransformer - end (186184, 20)
fit - DateTransformer - begin (186184, 20)
fit - DateTransformer - end (186184, 20)
transform - DateTransformer (186184, 20)
transform - DateTransformer - end (186184, 22)
fit - RobustScalerTransformer - begin (186184, 22)
fit - RobustScalerTransformer - end (186184, 22)
transform - RobustScalerTransformer - begin (186184, 22)
transform - RobustScalerTransformer - end (186184, 22)
transform - ManualFeatureSelectorTransformer (46547, 22)
transform - ManualFeatureSelect

c:\Users\Public\Anaconda\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (15) reached and the optimization hasn't converged yet.
  warnings.warn(


mlp
training:
transform - ManualFeatureSelectorTransformer (186184, 22)
transform - ManualFeatureSelectorTransformer - end (186184, 20)
transform - ImputerTransformer - begin (186184, 20)
transform - ImputerTransformer - end (186184, 20)
transform - DateTransformer (186184, 20)
transform - DateTransformer - end (186184, 22)
transform - RobustScalerTransformer - begin (186184, 22)
transform - RobustScalerTransformer - end (186184, 22)
Accuracy: (0.687244876036609,) | Precision:0.4592928420648628 | Recall:0.7444111815248241 | F1-score:0.568084380423979 | FP:45082 | FN:13148 | VP:38294 | VN:89660
over sampled training:
Accuracy: (0.7060307847590209,) | Precision:0.6906662087912088 | Recall:0.7463226017129032 | F1-score:0.7174165840295069 | FP:45039 | FN:34181 | VP:100561 | VN:89703
test:
Accuracy: (0.6887447096483125,) | Precision:0.4636804455326708 | Recall:0.744354527938343 | F1-score:0.5714116672583126 | FP:11171 | FN:3317 | VP:9658 | VN:22401


In [ ]:
data = pd.read_csv('data.csv', delimiter=';')

data.drop('Radiacao Global', axis=1, inplace=True)
data.drop('Cidade', axis=1, inplace=True)
data.drop('Codigo', axis=1, inplace=True)
data.drop('Latitude', axis=1, inplace=True)
data.drop('Longitude', axis=1, inplace=True)

data['Data'] = pd.to_datetime(data['Data'])
data['Ano'] = data['Data'].dt.year
data.drop('Data', axis=1, inplace=True)

counter = 0
for i in range(2001,2024):
    filtro = data.loc[(data['Ano']  == i)].drop('Ano', axis=1)
    counter += filtro.isna().any(axis=1).sum()
    print('ano-',i,':', filtro.isna().any(axis=1).sum())
print('any:',counter)

counter = 0
for i in range(2001,2024):
    filtro = data.loc[(data['Ano']  == i)].drop('Ano', axis=1)
    counter += filtro.isna().all(axis=1).sum()
    print('ano-',i,':', filtro.isna().all(axis=1).sum())
print('all:',counter)

counter = 0
for i in range(2001,2024):
    counter += data.loc[(data['Ano']  == i)]['Vai Chover Amanha'].isna().sum()
    print('ano-',i,':', data.loc[(data['Ano']  == i)]['Vai Chover Amanha'].isna().sum())
print('y:',counter)

In [588]:
data = pd.read_csv('data.csv', delimiter=';')

data = data.dropna()
data.reset_index(drop=True, inplace=True)

columns = ['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Radiacao Global']

counter = 0
for column in columns:
    q25, q75 = np.percentile(data[column], 25), np.percentile(data[column], 75)
    iqr = q75 - q25
    cut_off = iqr * 1.5
    lower_bound, upper_bound = q25 - cut_off, q75 + cut_off
    outliers = [x for x in data[column] if x < lower_bound or x > upper_bound]
    num_outliers = len(outliers)
    print(column,num_outliers)
    counter += num_outliers
print(counter)

counter = 0
for column in columns:
    z_scores = np.abs(stats.zscore(data[column]))
    threshold = 4
    outliers_mask = (z_scores > threshold)

    num_outliers = outliers_mask.sum()
    print(column,num_outliers)
    counter += outliers_mask.sum()
print(counter)

column = 'Precipitacao Total'
data = data[data[column] != 0]
counter = 0
q25, q75 = np.percentile(data[column], 25), np.percentile(data[column], 75)
iqr = q75 - q25
cut_off = iqr * 1.5
lower_bound, upper_bound = q25 - cut_off, q75 + cut_off
outliers = [x for x in data[column] if x < lower_bound or x > upper_bound]
num_outliers = len(outliers)
print(column,num_outliers)

z_scores = np.abs(stats.zscore(data[column]))
threshold = 4
outliers_mask = (z_scores > threshold)

num_outliers = outliers_mask.sum()
print(column,num_outliers)

Pressao Maxima 1305
Pressao Minima 1282
Temperatura Maxima 292
Temperatura Minima 1328
Temperatura Orvalho Maxima 2972
Temperatura Orvalho Minima 1498
Umidade Minima 0
Umidade Maxima 12532
Precipitacao Total 42323
Pressao Media 1453
Temperatura Media 659
Temperatura Orvalho Media 2221
Umidade Media 1729
Direcao Vento 1787
Rajada Maxima de Vento 7208
Vento Velocidade Media 5729
Radiacao Global 185
84503
Pressao Maxima 0
Pressao Minima 0
Temperatura Maxima 0
Temperatura Minima 2
Temperatura Orvalho Maxima 24
Temperatura Orvalho Minima 23
Umidade Minima 0
Umidade Maxima 1328
Precipitacao Total 3107
Pressao Media 0
Temperatura Media 0
Temperatura Orvalho Media 7
Umidade Media 42
Direcao Vento 0
Rajada Maxima de Vento 616
Vento Velocidade Media 615
Radiacao Global 69
5833
Precipitacao Total 8528
Precipitacao Total 942


In [49]:
data = pd.read_csv('data.csv', delimiter=';')

In [50]:
data.drop('Radiacao Global', axis=1, inplace=True)
data.drop('Cidade', axis=1, inplace=True)

In [ ]:
print(data.shape[0])
data = data.dropna()
data.reset_index(drop=True, inplace=True)
print(data.shape[0])
print(data.isna().sum())

In [ ]:
data['Data'] = pd.to_datetime(data['Data'])
data['Ano'] = data['Data'].dt.year
data['Mes'] = data['Data'].dt.month
data['Dia'] = data['Data'].dt.day
data.drop('Data', axis=1, inplace=True)
print(data.shape)

In [53]:
encoder = OneHotEncoder(sparse=False)
encoded_data = encoder.fit_transform(data[['Codigo']])
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(['Codigo']))
data = pd.concat([data.drop(['Codigo'], axis=1), encoded_df], axis=1)

In [54]:
scaler = RobustScaler()
data[['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima','Umidade Minima',
      'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 'Direcao Vento',
      'Rajada Maxima de Vento','Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia']] = scaler.fit_transform(
          data[['Pressao Maxima','Pressao Minima','Temperatura Maxima','Temperatura Minima','Temperatura Orvalho Maxima','Temperatura Orvalho Minima',
                'Umidade Minima', 'Umidade Maxima','Precipitacao Total', 'Pressao Media', 'Temperatura Media', 'Temperatura Orvalho Media', 'Umidade Media', 
                'Direcao Vento', 'Rajada Maxima de Vento', 'Vento Velocidade Media', 'Latitude', 'Longitude', 'Ano', 'Mes', 'Dia']])

In [55]:
X = data.drop('Vai Chover Amanha', axis=1)
y = data['Vai Chover Amanha']

In [ ]:
lof = LocalOutlierFactor(contamination=0.1)
yhat = lof.fit_predict(X)
mask = yhat != -1
X = X[mask]
y = y[mask]

In [ ]:
raw_X_train, X_test, raw_y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train_data = pd.concat([raw_X_train, raw_y_train], axis=1)

majority_class_data = train_data[train_data['Vai Chover Amanha'] == 'Nao']
minority_class_data = train_data[train_data['Vai Chover Amanha'] == 'Sim']
upsampled_miNaority_class = resample(minority_class_data, replace=True, n_samples=len(majority_class_data))
train_data = pd.concat([majority_class_data, upsampled_miNaority_class], axis=0)

balanced_X_train = train_data.drop('Vai Chover Amanha', axis=1)
balanced_y_train = train_data['Vai Chover Amanha']

print(train_data['Vai Chover Amanha'].value_counts())

In [ ]:
sm = SMOTE(random_state=42,k_neighbors=15)
print(raw_y_train.value_counts())
balanced_X_train, balanced_y_train = sm.fit_resample(raw_X_train, raw_y_train)
print(balanced_y_train.value_counts())

In [ ]:
sm = BorderlineSMOTE(random_state=42)
print(raw_y_train.value_counts())
balanced_X_train, balanced_y_train = sm.fit_resample(raw_X_train, raw_y_train)
print(balanced_y_train.value_counts())

In [ ]:
sm = SVMSMOTE(random_state=42)
print(raw_y_train.value_counts())
balanced_X_train, balanced_y_train = sm.fit_resample(raw_X_train, raw_y_train)
print(balanced_y_train.value_counts())

In [ ]:
sm = KMeansSMOTE(random_state=42)
print(raw_y_train.value_counts())
balanced_X_train, balanced_y_train = sm.fit_resample(raw_X_train, raw_y_train)

print(balanced_y_train.value_counts())

In [ ]:

print(balanced_X_train.shape)
selector = SelectKBest(k=20)
processed_X_train = selector.fit_transform(balanced_X_train, balanced_y_train)
processed_y_train = balanced_y_train.copy()
print(processed_X_train.shape)

In [ ]:
print(balanced_X_train.shape)
dt = DecisionTreeClassifier(splitter='best', criterion='gini')
selector = SequentialFeatureSelector(dt, n_features_to_select=20)
processed_X_train = selector.fit_transform(balanced_X_train, balanced_y_train)
processed_y_train = balanced_y_train.copy()
print(processed_X_train.shape)

In [ ]:
print(balanced_X_train.shape)
rf = RandomForestClassifier(class_weight='balanced', random_state=42)
selector = SelectFromModel(rf)
processed_X_train = selector.fit_transform(balanced_X_train, balanced_y_train)
processed_y_train = balanced_y_train.copy()
print(processed_X_train.shape)

In [ ]:
print(balanced_X_train.shape)
selector = SelectKBest(k=66)
processed_X_train = selector.fit_transform(balanced_X_train, balanced_y_train)
processed_y_train = balanced_y_train.copy()
print(processed_X_train.shape)